# 附三 基于 LangChain 自定义 Embeddings

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型的 Embeddings，内置了 OpenAI、LLAMA 等大模型 Embeddings 的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 Embeddings 类型，来提供强大的可扩展性。

在本部分，我们以智谱 AI 为例，讲述如何基于 LangChain 自定义 Embeddings，让我们基于 LangChain 搭建的应用能够支持百度文心、智谱 AI 等国内大模型生成的 Embeddings（讯飞星火目前还不支持）。

本部分涉及相对更多 LangChain、大模型调用的技术细节，有精力同学可以学习部署，如无精力可以直接使用后续代码来支持调用。

要实现自定义 Embeddings，需要定义一个自定义类继承自 LangChain 的 Embeddings 基类，然后定义三个函数：  
① _embed 方法，其接受一个字符串，并返回一个存放 Embeddings 的 List[float]，即模型的核心调用；  
② embed_query 方法，用于对单个字符串（query）进行 embedding。③ embed_documents 方法，用于对字符串列表（documents）进行 embedding。  

首先我们导入所需的第三方库：

In [1]:
from __future__ import annotations

import logging
from typing import Any, Dict, List, Optional

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator
from langchain.utils import get_from_dict_or_env

这里我们定义一个继承自 Embeddings 类的自定义 Embeddings 类：

In [ ]:
class ZhipuAIEmbeddings(BaseModel, Embeddings):
    """`Zhipuai Embeddings` embedding models."""

    zhipuai_api_key: Optional[str] = None
    """Zhipuai application apikey"""

在 Python 中，root_validator 是 Pydantic 模块中一个用于自定义数据校验的装饰器函数。root_validator 用于在校验整个数据模型之前对整个数据模型进行自定义校验，以确保所有的数据都符合所期望的数据结构。

root_validator 接收一个函数作为参数，该函数包含需要校验的逻辑。函数应该返回一个字典，其中包含经过校验的数据。如果校验失败，则抛出一个 ValueError 异常。

装饰器 root_validator 确保导入了相关的包和并配置了相关的 API_Key
这里取巧，在确保导入 zhipuai model 后直接将`zhipuai.model_api`绑定到 cliet 上，减少和其他 Embeddings 类的差异。


`values["client"] = zhipuai.model_api`

In [ ]:
    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        验证环境变量或配置文件中的zhipuai_api_key是否可用。

        Args:

            values (Dict): 包含配置信息的字典，必须包含 zhipuai_api_key 的字段
        Returns:

            values (Dict): 包含配置信息的字典。如果环境变量或配置文件中未提供 zhipuai_api_key，则将返回原始值；否则将返回包含 zhipuai_api_key 的值。
        Raises:

            ValueError: zhipuai package not found, please install it with `pip install
            zhipuai`
        """
        values["zhipuai_api_key"] = get_from_dict_or_env(
            values,
            "zhipuai_api_key",
            "ZHIPUAI_API_KEY",
        )

        try:
            import zhipuai
            zhipuai.api_key = values["zhipuai_api_key"]
            values["client"] = zhipuai.model_api

        except ImportError:
            raise ValueError(
                "Zhipuai package not found, please install it with "
                "`pip install zhipuai`"
            )
        return values


接下来我们重写 `_embed` 方法，调用远程 API 并解析 embedding 结果。

In [ ]:
    def _embed(self, texts: str) -> List[float]:
        """
        生成输入文本的 embedding。
        
        Args:
            texts (str): 要生成 embedding 的文本。

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表。
        """
        try:
            resp = self.client.invoke(
                model="text_embedding",
                prompt=texts
            )
        except Exception as e:
            raise ValueError(f"Error raised by inference endpoint: {e}")

        if resp["code"] != 200:
            raise ValueError(
                "Error raised by inference API HTTP code: %s, %s"
                % (resp["code"], resp["msg"])
            )
        embeddings = resp["data"]["embedding"]
        return embeddings

重写 embed_documents 方法，因为这里 `_embed` 已经定义好了，可以直接传入文本并返回结果即可。

In [ ]:
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding。
        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self._embed(text) for text in texts]

`embed_query` 是对单个文本计算 embedding 的方法，因为我们已经定义好对文档列表计算 embedding 的方法`embed_documents` 了，这里可以直接将单个文本组装成 list 的形式传给 `embed_documents`。

In [ ]:
    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding。
        
        Args:
            text (str): 要生成 embedding 的文本。

        Return:
            List [float]: 输入文本的 embedding，一个浮点数值列表。
        """
        resp = self.embed_documents([text])
        return resp[0]

可能大家会好奇，为什么要先定义`embed_documents`再用 `embed_query` 调用呢，不返过来呢，其实也是可以的，`embed_query` 单独请求也是可以的。

对于 `embed_documents` 可以加入一些内容处理后再请求 embedding，比如如果文档特别长，我们可以考虑对文档分段，防止超过最大 token 限制，这些都是可以的，靠大家发挥自己的主观能动性完善啦，这里只是给出一个简单的 demo。

目前只是实现了同步调用方式，后续优化中大家可以按需完善异步调用的方式，这里因为智谱官方不支持就没有实现。

通过上述步骤，我们就可以基于 LangChain 定义智谱 AI 生成 embedding 的调用方式了。我们将此代码封装在 zhipuai_embedding.py 文件中。